In [1]:
# TESTLIB_PATH = "/data/Alchemist-TestLib/target/testlib.dylib"
TESTLIB_PATH = "/Users/kai/Projects/AlLib/target/testlib.dylib"
# TESTLIB_PATH = "/usr/local/TestLib/target/testlib.so"
HOSTNAME = "localhost"
PORT = 24960

In [2]:
from alchemist import *
import numpy as np

als = AlchemistSession()        # Start Alchemist session

als.connect_to_alchemist(HOSTNAME, PORT)

Starting Alchemist session ... ready
Connecting to Alchemist at localhost:24960 ... 
          Client ID:                  0
          Session ID:                 0
          Command code:               1 (HANDSHAKE)
          Error code:                 0 (NONE)
          Message body length:        180
         -------------------------------------------------------------------------------
          BYTE                        4
          SHORT                       1234
          STRING                      ABCD
          DOUBLE                      1.11
          DOUBLE                      2.22
          MATRIX BLOCK                4x3 | [0 1 2 3] | [0 1 2]
          INT                         100000010
 
          Client ID:                  1
          Session ID:                 1
          Command code:               1 (HANDSHAKE)
          Error code:                 0 (NONE)
          Message body length:        19
         --------------------------------------------------

In [3]:
als.request_workers(3) 

Requesting 3 workers from Alchemist
 
          Client ID:                  1
          Session ID:                 1
          Command code:               11 (REQUEST_WORKERS)
          Error code:                 0 (NONE)
          Message body length:        3
         -------------------------------------------------------------------------------
          SHORT                       3
 
          Client ID:                  1
          Session ID:                 1
          Command code:               11 (REQUEST_WORKERS)
          Error code:                 0 (NONE)
          Message body length:        117
         -------------------------------------------------------------------------------
          SHORT                       3
          WORKER INFO                 
                                      ID:           1
                                      Hostname:     KaisMacBookPro.local
                                      Address:      0.0.0.0
                      

In [4]:
TestLib = als.load_library("TestLib", TESTLIB_PATH)

 
          Client ID:                  1
          Session ID:                 1
          Command code:               22 (LOAD_LIBRARY)
          Error code:                 0 (NONE)
          Message body length:        59
         -------------------------------------------------------------------------------
          STRING                      TestLib
          STRING                      /Users/kai/Projects/AlLib/target/testlib.dylib
 
          Client ID:                  1
          Session ID:                 1
          Command code:               22 (LOAD_LIBRARY)
          Error code:                 0 (NONE)
          Message body length:        2
         -------------------------------------------------------------------------------
          LIBRARY ID                  3
Library 'TestLib' successfully loaded.


In [5]:
A = np.random.rand(200,10)

A_handle = als.send_matrix(matrix=A, print_times=True, layout="VC_STAR")

Sending array info to Alchemist ...  
          Client ID:                  1
          Session ID:                 1
          Command code:               31 (SEND_MATRIX_INFO)
          Error code:                 0 (NONE)
          Message body length:        25
         -------------------------------------------------------------------------------
          STRING                      
          LONG                        200
          LONG                        10
          BYTE                        0
          BYTE                        11
 
          Client ID:                  1
          Session ID:                 1
          Command code:               31 (SEND_MATRIX_INFO)
          Error code:                 0 (NONE)
          Message body length:        45
         -------------------------------------------------------------------------------
          MATRIX INFO                 
                                      Name:                  
                      

In [6]:
in_byte = 9
in_char = 'y'
in_short = 9876
in_int = 987654321
in_long = 98765432123456789
in_float = 77.77777777
in_double = 88.88888888888888888
in_string = "test string"

TestLib.greet(in_byte, in_char, in_short, in_int, in_long, in_float, in_double, in_string)

List of input parameters:
    BYTE              in_byte
                      9
    CHAR              in_char
                      y
    SHORT             in_short
                      1348
    INT               in_int
                      987654321
    LONG              in_long
                      98765432123456789
    FLOAT             in_float
                      77.77777777
    DOUBLE            in_double
                      88.88888888888889
    STRING            in_string
                      test string
Alchemist started task 'greet' ...  
          Client ID:                  1
          Session ID:                 1
          Command code:               41 (RUN_TASK)
          Error code:                 0 (NONE)
          Message body length:        152
         -------------------------------------------------------------------------------
          LIBRARY ID                  3
          STRING                      greet
PPPP in_byte 33
          PARAMETER BYTE   

In [ ]:
rank = 5
S_handle, U_handle, V_handle = TestLib.truncated_svd(A_handle, rank)

In [ ]:
S = als.fetch_matrix(S_handle, print_times=True)

print(np.flipud(S).transpose())

In [ ]:
V = als.fetch_matrix(V_handle, print_times=True)
U = als.fetch_matrix(U_handle, print_times=True)
S = als.fetch_matrix(S_handle, print_times=True)

In [ ]:
u, s, vh = np.linalg.svd(A, full_matrices=True)

In [ ]:
print(np.linalg.norm(abs(V.T) - abs(np.flipud(vh[0:rank,:]))))
print(np.linalg.norm(abs(U) - abs(np.fliplr(u[:,0:rank]))))
print(np.linalg.norm(np.flip(S).T - s[0:rank]))

In [ ]:
als.stop()